In [ ]:
phenotypic_data = phenotypic_data.sample(frac = 1)
train, test = train_test_split(phenotypic_data, shuffle=True, test_size = 0.5)
cph.fit(train, duration_col="duration", event_col="is_hcm", fit_options = {"step_size":0.1})
hazard_scores = cph.predict_partial_hazard(test)

In [ ]:
    X = phenotypic_data[['sex','is_family_hist', 'age', 'principal_component_1', 'principal_component_4', 'prs_score', 'am_pathogenicity']].values
    y = phenotypic_data['is_hcm'].values
   
    for train_idx, test_idx in skf.split(X, y):
        train = phenotypic_data.iloc[train_idx]
        test = phenotypic_data.iloc[test_idx]
        train = train.drop(['var_index'], axis=1)
        test = test.drop(['var_index'], axis=1)

        # fit CPH and add hazard scores
        cph.fit(train, duration_col="duration", event_col="is_hcm", fit_options = {"step_size":0.1})
        hazard_scores_fold = cph.predict_partial_hazard(test)
        hazard_scores.extend(hazard_scores_fold)


In [ ]:
    phenotypic_data = phenotypic_data.sample(frac=1)
    variant_data = variant_data.sample(frac=1)
    setA, setB, setC, setD, setE = np.array_split(variant_data['var_index'].to_list(), 5)

    dfs = [setA, setB, setC, setD, setE]
    for i in range(len(dfs)):
        dfs[i] = phenotypic_data[phenotypic_data['var_index'].isin(dfs[i])]

    # drop var index


    # fit CPH and add hazard scores
    new_dfs = copy.deepcopy(dfs)

    for i in range(len(dfs)):
        concatenated_df = pd.concat([df for idx, df in enumerate(dfs) if idx != i], ignore_index=True)
        cph.fit(concatenated_df, duration_col="duration", event_col="is_hcm", fit_options = {"step_size":0.1})
        hazard_scores_fold = cph.predict_partial_hazard(dfs[i])
        new_dfs[i]['hazard'] = hazard_scores_fold
        


    phenotypic_data = pd.concat([df for idx, df in enumerate(new_dfs)])
    print(phenotypic_data)

In [ ]:
    X = phenotypic_data.drop(['var_index', 'is_hcm', 'duration'], axis = 1)
    y = phenotypic_data[['duration', 'is_hcm']]

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Fit the CoxPH model on the training data
        cph.fit(pd.concat([X_train, y_train], axis=1), duration_col='duration', event_col='is_hcm')

        # Predict and evaluate on the test data
        cph_score = cph.score(pd.concat([X_test, y_test], axis=1), scoring_method='concordance_index')
        hazard_scores_fold = cph.predict_partial_hazard(pd.concat([X_test, y_test], axis = 1))
        hazard_scores.extend(hazard_scores_fold)
        print("Concordance Index (C-Index) on test set:", cph_score)        

In [ ]:
cph = CoxPHFitter()
kf = KFold(n_splits=4)
for train_idx, test_idx in kf.split(phenotypic_data):
    train = phenotypic_data.iloc[train_idx]
    test = phenotypic_data.iloc[test_idx]
    print(train, test)
    train = train.drop(['var_index'], axis=1)
    test = test.drop(['var_index'], axis=1)

    # fit CPH and add hazard scores
    cph.fit(train, duration_col="duration", event_col="is_hcm", fit_options = {"step_size":0.1})
    hazard_scores_fold = cph.predict_partial_hazard(test)
    hazard_scores.extend(hazard_scores_fold)


        